###### Imports and Settings

In [3]:
import pandas as pd
import numpy as np
import requests
import pickle
from collections import deque
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [5]:
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']

In [6]:
def percent(x, y):
    return (x/y)*100

In [7]:
import geodict
GNRC = geodict.GNRC
KY = geodict.KY
censusplaces = geodict.censusplaces

## Read In Data Guide

In [15]:
dataguide = pd.read_csv('../../Data Guides/DATA GUIDE ACS1YR.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [16]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
#dg2 = dataguide[dataguide['ID'].between(47, 92)]

In [17]:
years = ['2017', '2018', '2019', '2021']

In [20]:
results = []
for year in years:
    dataguide = dg1
    url_str= 'https://api.census.gov/data/{}/acs/acs1?key='.format(year)+api_key
    head1 = 'NAME' 
    head2 = 'GEO_ID'
    tail_cols1 = 'StateFIPS'
    tail_cols2 = 'GeoFIPS'
    var_list = list(dataguide['Variable']) #make variables list
    var_list = deque(var_list)
    var_list.appendleft(head2)
    var_list.appendleft(head1)
    col_list = list(dataguide['Column Name']) #make columns list
    col_list.append(tail_cols1)
    col_list.append(tail_cols2)
    col_list = deque(col_list)
    col_list.appendleft(head2)
    col_list.appendleft(head1)
    predicates= {} #call for all counties in state of TN
    get_vars= var_list
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "county:*"
    predicates["in"]= "state:47"                                                             
    data = requests.get(url_str, params= predicates)                                                                
    col_names = col_list
    df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
    df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
    predicates= {} #call for all counties in state of KY
    get_vars= var_list
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "county:*"
    predicates["in"]= "state:21"                                                             
    data = requests.get(url_str, params= predicates)                                                                
    col_names = col_list
    kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
    kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
    df = pd.concat([df, kycos], axis = 0)
    predicates= {} #call for all places in TN
    get_vars= var_list
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "place:*"
    predicates["in"]= "state:47"
    data= requests.get(url_str, params= predicates)
    col_names = col_list
    places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
    places=places.loc[places['GEO_ID'].isin(censusplaces)] #filter for places in the region
    df = pd.concat([df, places], axis = 0)
    predicates= {} #call for all places in KY
    get_vars= var_list
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "place:*"
    predicates["in"]= "state:21"
    data= requests.get(url_str, params= predicates)
    col_names = col_list
    places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
    places=places.loc[places['GEO_ID'].isin(censusplaces)] #filter for places of concern in KY
    df = pd.concat([df, places], axis = 0)
    col_list.remove(tail_cols2) #adjust the column list for different formats of geos
    predicates= {} #call for state of TN
    get_vars= var_list
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "state:47"
    data= requests.get(url_str, params= predicates)
    col_names = col_list
    state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
    state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
    df = pd.concat([df, state], axis = 0)
    predicates= {} #call for US
    get_vars= var_list
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "us:*"
    data= requests.get(url_str, params= predicates)
    col_names = col_list
    national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
    national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
    df = pd.concat([df, national], axis = 0)
    df['Year'] = '{}'.format(year)
    results.append(df)
new_df = pd.concat(results)#, axis = 1)
print('Okay Finished')

Okay Finished


In [22]:
new_df.tail()

,NAME,GEO_ID,geo_total_series,geo_samehouse_total,geo_movedsamecounty_total,geo_moveddifferentcountysamestate_total,geo_moveddifferentstate_total,geo_movedabroad_total,StateFIPS,GeoFIPS,Year
1,Nashville-Davidson metropolitan government (ba...,1600000US4752006,668586,528965,70135,31379,33447,4660,47,52006,2021
5,"Franklin city, Tennessee",1600000US4727740,None,None,None,None,None,None,47,27740,2021
7,"Clarksville city, Tennessee",1600000US4715160,None,None,None,None,None,None,47,15160,2021
0,Tennessee,0400000US47,6899165,5955858,462187,239146,220591,21383,47,0,2021
0,United States,0100000US,328464538,286552923,21878668,10698453,7859837,1474657,1,0,2021


In [27]:
data = new_df
data = data.set_index(['NAME', 'GEO_ID'])
data.head(2)

,,geo_total_series,geo_samehouse_total,geo_movedsamecounty_total,geo_moveddifferentcountysamestate_total,geo_moveddifferentstate_total,geo_movedabroad_total,StateFIPS,GeoFIPS,Year
NAME,GEO_ID,,,,,,,,,
"Maury County, Tennessee",0500000US47119,None,None,None,None,None,None,47,119,2017
"Robertson County, Tennessee",0500000US47147,None,None,None,None,None,None,47,147,2017


In [28]:
data = data.transpose().drop(columns = [('Maury County, Tennessee', '0500000US47119'), ('Robertson County, Tennessee', '0500000US47147'),
                                ('Wilson County, Tennessee', '0500000US47189'), ('Franklin city, Tennessee', '1600000US4727740')])
data = data.transpose()
data.head()

,,geo_total_series,geo_samehouse_total,geo_movedsamecounty_total,geo_moveddifferentcountysamestate_total,geo_moveddifferentstate_total,geo_movedabroad_total,StateFIPS,GeoFIPS,Year
NAME,GEO_ID,,,,,,,,,
"Sumner County, Tennessee",0500000US47165,181562,153887,13703,7551,5816,605,47,165,2017
"Williamson County, Tennessee",0500000US47187,223906,188492,9044,15925,10050,395,47,187,2017
"Davidson County, Tennessee",0500000US47037,680507,551997,76528,16492,30785,4705,47,037,2017
"Montgomery County, Tennessee",0500000US47125,195777,145895,22840,4073,20447,2522,47,125,2017
"Rutherford County, Tennessee",0500000US47149,312925,265690,20429,17054,8791,961,47,149,2017


In [29]:
numcols = list(data.columns)
data[numcols] = data[numcols].astype(float)

### Not aggregating for regions or unincorporated areas as data is only available for select geographies.

In [30]:
coldropper = data
data = data.reset_index()

In [31]:
#calculations
#geographic mobility
data['GeoMobility:Total'] = data['geo_total_series']
data['GeoMobility:Same House'] = data['geo_samehouse_total']
data['GeoMobility%:Same House'] = percent(data['GeoMobility:Same House'],data['GeoMobility:Total'])
data['GeoMobility:Moved Same County'] = data['geo_movedsamecounty_total']
data['GeoMobility%:Moved Same County'] = percent(data['GeoMobility:Moved Same County'],data['GeoMobility:Total'])
data['GeoMobility:Moved Different County Same State'] = data['geo_moveddifferentcountysamestate_total']
data['GeoMobility%:Moved Different County Same State'] = percent(data['GeoMobility:Moved Different County Same State'],data['GeoMobility:Total'])
data['GeoMobility:Moved Different State'] = data['geo_moveddifferentstate_total']
data['GeoMobility%:Moved Different State'] = percent(data['GeoMobility:Moved Different State'],data['GeoMobility:Total'])
data['GeoMobility:Moved From Abroad'] = data['geo_movedabroad_total']
data['GeoMobility%:Moved From Abroad'] = percent(data['GeoMobility:Moved From Abroad'],data['GeoMobility:Total'])

In [32]:
cols = coldropper.columns
data = data.drop(columns = cols)
data.head(3)

,NAME,GEO_ID,GeoMobility:Total,GeoMobility:Same House,GeoMobility%:Same House,GeoMobility:Moved Same County,GeoMobility%:Moved Same County,GeoMobility:Moved Different County Same State,GeoMobility%:Moved Different County Same State,GeoMobility:Moved Different State,GeoMobility%:Moved Different State,GeoMobility:Moved From Abroad,GeoMobility%:Moved From Abroad
0,"Sumner County, Tennessee",0500000US47165,181562.0,153887.0,84.757273,13703.0,7.547284,7551.0,4.158910,5816.0,3.203313,605.0,0.333220
1,"Williamson County, Tennessee",0500000US47187,223906.0,188492.0,84.183541,9044.0,4.039195,15925.0,7.112360,10050.0,4.488491,395.0,0.176413
2,"Davidson County, Tennessee",0500000US47037,680507.0,551997.0,81.115551,76528.0,11.245733,16492.0,2.423487,30785.0,4.523833,4705.0,0.691396


In [34]:
data.to_feather('../../Outputs/2017_2018_2019_2021ACS1YR')